In [2]:
import databaseOps, fields, matching
import numpy as np
import pandas as pd
import json, sqlite3
from bs4 import BeautifulSoup, Tag

In [3]:
conn = databaseOps.connectToDb()
cur = conn.cursor()

In [4]:
"""
What to predict on??
- naive: feed all header fields
"""
# Head table
df = pd.read_sql_query("select * from form4Head;", conn)
df.head()

,accNum,headId,rowNumber,documentType,publicDocCount,periodOfReport,filedDate,changedDate,schemaVersion,dateOfOriginalSubmission,...,rptOwnerStateDescription,rptOwnerMailStreet1,rptOwnerMailStreet2,rptOwnerMailCity,rptOwnerMailState,rptOwnerMailZip,rptOwnerisDirector,rptOwnerisOfficer,rptOwnerisTenPercentOwner,rptOwnerisOther
0,66740/0001127602-16-065866,66740/0001127602-16-065866-1,1,4,1,2016-11-01,20161102,20161102,X0306,None,...,,3M CENTER,None,ST. PAUL,MN,55144-1000,0,1,0,0
1,66740/0001127602-16-066334,66740/0001127602-16-066334-1,1,4,1,2016-11-04,20161108,20161108,X0306,None,...,,3M CENTER - OFFICE OF GENERAL COUNSEL,None,ST. PAUL,MN,55144-1000,0,1,0,0
2,66740/0001127602-16-066337,66740/0001127602-16-066337-1,1,4,1,2016-11-04,20161108,20161108,X0306,None,...,,3M CENTER - OFFICE OF GENERAL COUNSEL,None,ST. PAUL,MN,55144-1000,0,1,0,0
3,66740/0001127602-16-066578,66740/0001127602-16-066578-1,1,4,1,2016-11-08,20161109,20161109,X0306,None,...,,DEERE COMPANY,ONE JOHN DEERE PLACE,MOLINE,IL,61265,1,0,0,0
4,66740/0001127602-16-066589,66740/0001127602-16-066589-1,1,4,1,2016-11-08,20161109,20161109,X0306,None,...,,None,None,None,None,None,1,0,0,0


In [5]:
fours = df[df['documentType']=='4']['accNum'].values
fourAs = df[df['documentType']=='4/A']['accNum'].values
uniqueFours = np.unique(fours)
uniqueFourAs = np.unique(fourAs)

In [8]:
### Find changes for 4/a idx
# changeDic = matching.get4ATo4Changes(uniqueFourAs)
    
with open('2016Q4_2017Q4_3MCOChanges.json') as data_file:
    changeDic = json.load(data_file)
    
# matches = [(key, changeDic[key]['accNum']) for key in list(changeDic.keys())] #[(4a, 4)]

In [9]:
labels = np.zeros(len(fours))
fourwMatch = [changeDic[key]['accNum'] for key in list(changeDic.keys())]
labels[np.where(np.in1d(fours, fourwMatch))[0]] = 1

In [7]:
df2 = df[df['documentType']=='4']
df2 = df2[['changedDate', 'filedDate', 'publicDocCount', 'issuerCik', 'rptOwnerCik', 'rptOwnerisDirector', 'rptOwnerisOfficer', 'rptOwnerisOther', 'rptOwnerisTenPercentOwner', 'documentType']] 
#'dateOfOriginalSubmission' mostly None?
# 'notSubjectToSection16' NaN?
# TODO 'issuerFiscalYrEnd' change None to something

### TODO: categorize issuerCik/rptOwnerCik issuerMailZip, 
headData = df2.values

In [9]:
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

names = ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Gaussian Process",
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes"]#, "QDA"]

classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1),
    AdaBoostClassifier(),
    GaussianNB()]#,QuadraticDiscriminantAnalysis()]

In [ ]:
# need to use kfold validation bc sets are so small
from sklearn.model_selection import KFold

kf = KFold(n_splits=5)
kf.get_n_splits(headData)
for train_idx, valid_idx in kf.split(headData):
    train_data, valid_data = headData[train_idx], headData[valid_idx]
    train_label, valid_label = labels[train_idx], labels[valid_idx]

    for name, clf in zip(names, classifiers):
        clf.fit(train_data, train_label)
        train_score = clf.score(train_data, train_label)
        valid_score = clf.score(valid_data, valid_label)
        print(name, train_score, valid_score)
        
    print('\n')

Nearest Neighbors 0.8441558441558441 0.65
Linear SVM 0.7792207792207793 0.65
RBF SVM 1.0 1.0
Gaussian Process 1.0 1.0
Decision Tree 1.0 0.6
Random Forest 1.0 0.6


/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Neural Net 0.7792207792207793 1.0
AdaBoost 1.0 0.6
Naive Bayes 0.8311688311688312 0.75


Nearest Neighbors 0.987012987012987 0.2
Linear SVM 0.987012987012987 0.2
RBF SVM 1.0 0.2
Gaussian Process 1.0 0.2
Decision Tree 1.0 0.25
Random Forest 1.0 0.2


/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Neural Net 0.987012987012987 0.2
AdaBoost 1.0 0.25
Naive Bayes 1.0 0.2


Nearest Neighbors 0.8589743589743589 0.8947368421052632


In [13]:
# nonderive transac table
nonderive = pd.read_sql_query("select * from form4DT;", conn)
nonderive

,accNum,dTId,rowNumber,securityTitle,conversionOrExercisePrice,transactionDate,transactionFormType,transactionCode,equitySwapInvolved,transactionShares,...,transactionAcquiredDisposedCode,exerciseDate,expirationDate,underlyingSecurityTitle,underlyingSecurityShares,sharesOwnedFollowingTransaction,directOrIndirectOwnership,type,documentType,footNoteId
0,66740/0001127602-16-065866,66740/0001127602-16-065866-1,1,<value>Restricted Stock Units</value>,"<footnoteId id=""F2""/>",<value>2016-11-01</value>,4,M,0,<value>11205</value>,...,<value>D</value>,"<footnoteId id=""F3""/>","<footnoteId id=""F3""/>",<value>Common Stock</value>,<value>11205</value>,<value>0</value>,<value>D</value>,transaction,4,None
1,66740/0001127602-16-067517,66740/0001127602-16-067517-1,1,<value>Non-qualified Stock Option (Right to Bu...,<value>84.78</value>,<value>2016-11-15</value>,4,M,0,<value>2952</value>,...,<value>D</value>,<value>2008-05-08</value>,<value>2017-05-08</value>,<value>Common Stock</value>,<value>2952</value>,<value>0</value>,<value>D</value>,transaction,4,None
2,66740/0001127602-16-067701,66740/0001127602-16-067701-1,1,<value>Non-qualified Stock Option (Right to Bu...,<value>77.18</value>,<value>2016-11-17</value>,4,M,0,<value>10424</value>,...,<value>D</value>,<value>2009-05-13</value>,<value>2018-05-13</value>,<value>Common Stock</value>,<value>10424</value>,<value>0</value>,<value>D</value>,transaction,4,None
3,66740/0001127602-16-069469,66740/0001127602-16-069469-1,1,<value>Non-qualified Stock Option (Right to Bu...,<value>84.78</value>,<value>2016-12-07</value>,4,M,0,"<value>436</value><footnoteId id=""F1""/>",...,<value>D</value>,<value>2008-05-08</value>,<value>2017-05-08</value>,<value>Common Stock</value>,<value>436</value>,<value>0</value>,<value>D</value>,transaction,4,None
4,66740/0001127602-16-069967,66740/0001127602-16-069967-1,1,<value>Non-qualified Stock Option (Right to Bu...,<value>84.78</value>,<value>2016-12-13</value>,4,M,0,<value>4036</value>,...,<value>D</value>,<value>2008-05-08</value>,<value>2017-05-08</value>,<value>Common Stock</value>,<value>4036</value>,<value>0</value>,<value>D</value>,transaction,4,None
5,66740/0001127602-17-004736,66740/0001127602-17-004736-1,1,<value>Non-qualified Stock Option (Right to Bu...,<value>175.76</value>,<value>2017-02-07</value>,4,A,0,<value>39318</value>,...,<value>A</value>,<value>2018-02-07</value>,<value>2027-02-08</value>,<value>Common Stock</value>,<value>39318</value>,<value>39318</value>,<value>D</value>,transaction,4,None
6,66740/0001127602-17-004744,66740/0001127602-17-004744-1,1,<value>Non-qualified Stock Option (Right to Bu...,<value>175.76</value>,<value>2017-02-07</value>,4,A,0,<value>48264</value>,...,<value>A</value>,<value>2018-02-07</value>,<value>2027-02-08</value>,<value>Common Stock</value>,<value>48264</value>,<value>48264</value>,<value>D</value>,transaction,4,None
7,66740/0001127602-17-004746,66740/0001127602-17-004746-1,1,<value>Non-qualified Stock Option (Right to Bu...,<value>175.76</value>,<value>2017-02-07</value>,4,A,0,<value>48264</value>,...,<value>A</value>,<value>2018-02-07</value>,<value>2027-02-08</value>,<value>Common Stock</value>,<value>48264</value>,<value>48264</value>,<value>D</value>,transaction,4,None
8,66740/0001127602-17-004748,66740/0001127602-17-004748-1,1,<value>Non-qualified Stock Option (Right to Bu...,<value>175.76</value>,<value>2017-02-07</value>,4,A,0,<value>30623</value>,...,<value>A</value>,<value>2018-02-07</value>,<value>2027-02-08</value>,<value>Common Stock</value>,<value>30623</value>,<value>30623</value>,<value>D</value>,transaction,4,None
9,66740/0001127602-17-004750,66740/0001127602-17-004750-1,1,<value>Non-qualified Stock Option (Right to Bu...,<value>175.76</value>,<value>2017-02-07</value>,4,A,0,<value>78252</value>,...,<value>A</value>,<value>2018-02-07</value>,<value>2027-02-08</value>,<value>Common Stock</value>,<value>78252</value>,<value>78252</value>,<value>D</value>,transaction,4,None


In [99]:
nonderive2 = nonderive[nonderive['documentType']=='4']
# df2 = df2[['changedDate', 'filedDate', 'publicDocCount', 'issuerCik', 'rptOwnerCik', 'rptOwnerisDirector', 'rptOwnerisOfficer', 'rptOwnerisOther', 'rptOwnerisTenPercentOwner', 'documentType']]

headData = df2.values

array([['66740/0001127602-17-004523', '66740/0001127602-17-004523-1',
        '1', ..., '0', '0', '0'],
       ['66740/0001127602-17-004527', '66740/0001127602-17-004527-1',
        '1', ..., '0', '0', '0'],
       ['66740/0001127602-17-004736', '66740/0001127602-17-004736-1',
        '1', ..., '1', '0', '0'],
       ...,
       ['66740/0001127602-17-033806', '66740/0001127602-17-033806-1',
        '1', ..., '1', '0', '0'],
       ['66740/0001127602-17-033816', '66740/0001127602-17-033816-1',
        '1', ..., '1', '0', '0'],
       ['66740/0001127602-17-033846', '66740/0001127602-17-033846-1',
        '1', ..., '1', '0', '0']], dtype='<U47')

In [91]:
for col in df2.columns:
    if df2[col].dtype != int:
        df2[col] = df2[col].astype(str)
df2.dtypes

accNum                            object
headId                            object
rowNumber                          int64
documentType                      object
publicDocCount                     int64
periodOfReport                    object
filedDate                         object
changedDate                       object
schemaVersion                     object
dateOfOriginalSubmission          object
notSubjectToSection16             object
issuerName                        object
issuerCik                         object
issuerIndustrialClassification    object
issuerIrs                         object
issuerIncorpState                 object
issuerFiscalYrEnd                 object
issuerBusinessStreet1             object
issuerBusinessCity                object
issuerBusinessState               object
issuerBusinessZip                 object
issuerBusinessPhone               object
issuerMailStreet1                 object
issuerMailStreet2                 object
issuerMailCity  

In [90]:
dtype(df2['accNum'])

NameError: name 'dtype' is not defined

In [57]:
headData = df[df['accNum'].isin(fourwMatch)].values

17.0